### MOVIES DATA EXTRACTION:###

In [ ]:
#!pip install tmdbsimple

import requests
import tmdbsimple as tmdb
import pandas as pd

  Obtaining dependency information for tmdbsimple from https://files.pythonhosted.org/packages/6c/dd/ade05d202db728b23e54aa0959622d090776023917e7308c1b2469a07b76/tmdbsimple-2.9.1-py3-none-any.whl.metadata


 Check what's the movie info that we can download per movie observation

In [ ]:
import tmdbsimple as tmdb

# Replace with your actual TMDB API key
tmdb.API_KEY = '8db08b812689d86a7bf90611bf5a1eee'

# Use an example movie ID (550 is Fight Club)
movie_id = 550

movie = tmdb.Movies(movie_id)
data = movie.info()

# Print out all the available keys in the movie details response
print("Available movie fields (columns):")
for key in data:
    print(key)

Available movie fields (columns):
adult
backdrop_path
belongs_to_collection
budget
genres
homepage
id
imdb_id
origin_country
original_language
original_title
overview
popularity
poster_path
production_companies
production_countries
release_date
revenue
runtime
spoken_languages
status
tagline
title
video
vote_average
vote_count


Some of the fields (like genres, production companies, spoken languages) are nested structures. This will require additional code to explore and flatten the nested keys to have every subfield as an addittional column.

In [ ]:
### TRIAL HOW TO DOWNLOAD MOVIES

import csv
import tmdbsimple as tmdb


def fetch_movie_data(movie_id):
    """Fetch detailed movie info and credits for a given movie ID."""
    movie = tmdb.Movies(movie_id)
    try:
        details = movie.info()
        credits = movie.credits()
    except Exception as e:
        print(f"Error fetching data for movie ID {movie_id}: {e}")
        return None

    # Extract genres and join into a comma-separated string
    genres = [genre['name'] for genre in details.get('genres', [])]
    genres_str = ", ".join(genres)

    # Extract the top 5 cast members and join their names
    cast_list = credits.get('cast', [])
    cast_names = [member['name'] for member in cast_list[:5]]
    cast_str = ", ".join(cast_names)

    return {
        'movie_id': details.get('id'),
        'title': details.get('title'),
        'release_date': details.get('release_date'),
        'genres': genres_str,
        'cast': cast_str,
        'revenue': details.get('revenue'),
        'budget': details.get('budget'),
        'runtime': details.get('runtime'),
        'original_language': details.get('original_language'),
        'popularity': details.get('popularity'),
        'vote_average': details.get('vote_average'),
        'vote_count': details.get('vote_count'),
        'overview': details.get('overview')
    }

def main():
    output_file = 'tmdb_movies.csv'
    fieldnames = [
        'movie_id', 'title', 'release_date', 'genres', 'cast',
        'revenue', 'budget', 'runtime', 'original_language',
        'popularity', 'vote_average', 'vote_count', 'overview'
    ]

    with open(output_file, mode='w', newline='', encoding='utf-8') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()

        discover = tmdb.Discover()
        # Adjust total_pages as needed to gather more movies.
        total_pages = 5
        for page in range(1, total_pages + 1):
            print(f"Fetching page {page} of movies...")
            try:
                discover_response = discover.movie(page=page)
            except Exception as e:
                print(f"Error fetching discover page {page}: {e}")
                continue

            movies = discover_response.get('results', [])
            for movie in movies:
                movie_id = movie.get('id')
                movie_data = fetch_movie_data(movie_id)
                if movie_data:
                    writer.writerow(movie_data)
                    print(f"Written movie: {movie_data.get('title')}")

if __name__ == '__main__':
    main()


Fetching page 1 of movies...
Written movie: The Gorge
Written movie: Mufasa: The Lion King
Written movie: Flight Risk
Written movie: Moana 2
Written movie: Sonic the Hedgehog 3
Written movie: Amaran
Written movie: Captain America: Brave New World
Written movie: Panda Plan
Written movie: Companion
Written movie: My Fault: London
Written movie: Kraven the Hunter
Written movie: Paddington in Peru
Written movie: Dog Man
Written movie: Death Whisperer 2
Written movie: The Island
Written movie: Alarum
Written movie: Venom: The Last Dance
Written movie: The Brutalist
Written movie: Gladiator II
Written movie: Sniper: The Last Stand
Fetching page 2 of movies...
Written movie: Back in Action
Written movie: Dhoom Dhaam
Written movie: Ball Red Daughter-in-law
Written movie: Devara: Part 1
Written movie: Wolf Man
Written movie: Elevation
Written movie: The Gardener
Written movie: Solo Leveling -ReAwakening-
Written movie: Le clitoris
Written movie: Star Trek: Section 31
Written movie: Nosferatu
Wr



When designing your database schema, it's helpful to normalize your data by moving multi-valued and relational attributes into separate tables. Based on the TMDB movie details structure, here are some recommendations:

Genres:
Movies often belong to multiple genres. Create a separate Genres table and a join table (e.g., MovieGenres) to relate each movie to its genres.

Production Companies:
A movie can involve several production companies, and the same company might produce many movies. Having a ProductionCompanies table (plus a join table like MovieProductionCompanies) makes it easier to query and update company information without duplicating data.

Production Countries:
Similarly, since movies can be associated with multiple countries, a ProductionCountries table with a join table can help manage this many-to-many relationship.

Spoken Languages:
Movies can feature several spoken languages. A dedicated SpokenLanguages table with a linking table (e.g., MovieLanguages) keeps this relationship normalized.

Collections (Belongs_to_collection):
If you want to explore movie series or collections in more detail, a separate Collections table can be useful. This table can store additional information about the collection beyond just the name.

Cast and Crew (if extended):
Although not in your current list, if you decide to include cast/crew details later, these should definitely reside in separate tables (like Cast and Crew with their respective join tables) because a single movie can have many cast members, and each actor might appear in multiple movies.

In [ ]:
## TMDB’s Discover endpoint typically limits results to 500 pages. so better filter by dates first to handle each section of movies individually

In [ ]:
# trial only extract first 100 movies

In [ ]:
import csv
import time
import tmdbsimple as tmdb

# Set your TMDB API key
tmdb.API_KEY = '8db08b812689d86a7bf90611bf5a1eee'  # Replace with your actual API key

# -----------------------------
# Data Processing Functions
# -----------------------------
def get_movie_details(movie_id):
    """Fetch detailed information for a movie by its ID."""
    movie = tmdb.Movies(movie_id)
    try:
        details = movie.info()
        return details
    except Exception as e:
        print(f"Error retrieving movie {movie_id}: {e}")
        return None

def process_main_data(details):
    """Extract single-valued (non-normalized) fields for the main movies table."""
    return {
        "id": details.get("id"),
        "adult": details.get("adult"),
        "backdrop_path": details.get("backdrop_path"),
        "budget": details.get("budget"),
        "homepage": details.get("homepage"),
        "imdb_id": details.get("imdb_id"),
        "original_language": details.get("original_language"),
        "original_title": details.get("original_title"),
        "overview": details.get("overview"),
        "popularity": details.get("popularity"),
        "poster_path": details.get("poster_path"),
        "release_date": details.get("release_date"),
        "revenue": details.get("revenue"),
        "runtime": details.get("runtime"),
        "status": details.get("status"),
        "tagline": details.get("tagline"),
        "title": details.get("title"),
        "video": details.get("video"),
        "vote_average": details.get("vote_average"),
        "vote_count": details.get("vote_count")
    }

def process_collection(details):
    """Extract collection info (belongs_to_collection) into its own record."""
    coll = details.get("belongs_to_collection")
    if coll:
        return {
            "movie_id": details.get("id"),
            "collection_id": coll.get("id"),
            "collection_name": coll.get("name")
        }
    return None

def process_genres(details):
    """Extract the list of genres for a movie."""
    genres_list = details.get("genres", [])
    rows = []
    movie_id = details.get("id")
    for genre in genres_list:
        rows.append({
            "movie_id": movie_id,
            "genre_id": genre.get("id"),
            "genre_name": genre.get("name")
        })
    return rows

def process_production_companies(details):
    """Extract production companies for a movie."""
    companies = details.get("production_companies", [])
    rows = []
    movie_id = details.get("id")
    for comp in companies:
        rows.append({
            "movie_id": movie_id,
            "company_id": comp.get("id"),
            "company_name": comp.get("name"),
            "origin_country": comp.get("origin_country")
        })
    return rows

def process_production_countries(details):
    """Extract production countries for a movie."""
    countries = details.get("production_countries", [])
    rows = []
    movie_id = details.get("id")
    for country in countries:
        rows.append({
            "movie_id": movie_id,
            "iso_3166_1": country.get("iso_3166_1"),
            "country_name": country.get("name")
        })
    return rows

def process_spoken_languages(details):
    """Extract spoken languages for a movie."""
    languages = details.get("spoken_languages", [])
    rows = []
    movie_id = details.get("id")
    for lang in languages:
        rows.append({
            "movie_id": movie_id,
            "iso_639_1": lang.get("iso_639_1"),
            "language_name": lang.get("name")
        })
    return rows

# -----------------------------
# Main ETL Process
# -----------------------------
def main():
    # Define output CSV filenames
    main_file = "tmdb_movies_main.csv"
    collection_file = "tmdb_movie_collection.csv"
    genres_file = "tmdb_movie_genres.csv"
    production_companies_file = "tmdb_movie_production_companies.csv"
    production_countries_file = "tmdb_movie_production_countries.csv"
    spoken_languages_file = "tmdb_movie_spoken_languages.csv"

    # Define CSV headers for each file
    main_headers = [
        "id", "adult", "backdrop_path", "budget", "homepage", "imdb_id",
        "original_language", "original_title", "overview", "popularity",
        "poster_path", "release_date", "revenue", "runtime", "status",
        "tagline", "title", "video", "vote_average", "vote_count"
    ]
    collection_headers = ["movie_id", "collection_id", "collection_name"]
    genres_headers = ["movie_id", "genre_id", "genre_name"]
    production_companies_headers = ["movie_id", "company_id", "company_name", "origin_country"]
    production_countries_headers = ["movie_id", "iso_3166_1", "country_name"]
    spoken_languages_headers = ["movie_id", "iso_639_1", "language_name"]

    # Open CSV files for writing
    with open(main_file, mode="w", newline="", encoding="utf-8") as main_csv, \
         open(collection_file, mode="w", newline="", encoding="utf-8") as coll_csv, \
         open(genres_file, mode="w", newline="", encoding="utf-8") as genres_csv, \
         open(production_companies_file, mode="w", newline="", encoding="utf-8") as prod_comp_csv, \
         open(production_countries_file, mode="w", newline="", encoding="utf-8") as prod_countries_csv, \
         open(spoken_languages_file, mode="w", newline="", encoding="utf-8") as spoken_csv:

        main_writer = csv.DictWriter(main_csv, fieldnames=main_headers)
        coll_writer = csv.DictWriter(coll_csv, fieldnames=collection_headers)
        genres_writer = csv.DictWriter(genres_csv, fieldnames=genres_headers)
        prod_comp_writer = csv.DictWriter(prod_comp_csv, fieldnames=production_companies_headers)
        prod_countries_writer = csv.DictWriter(prod_countries_csv, fieldnames=production_countries_headers)
        spoken_writer = csv.DictWriter(spoken_csv, fieldnames=spoken_languages_headers)

        # Write headers to each CSV file
        main_writer.writeheader()
        coll_writer.writeheader()
        genres_writer.writeheader()
        prod_comp_writer.writeheader()
        prod_countries_writer.writeheader()
        spoken_writer.writeheader()

        # We'll use only one date range to simplify and then only process 100 movies.
        date_ranges = [
            {"gte": "1900-01-01", "lte": "1950-12-31"}
        ]

        movies_processed = 0  # Counter for processed movies

        # Loop through each date range partition
        for dr in date_ranges:
            print(f"Processing movies released from {dr['gte']} to {dr['lte']}")
            discover = tmdb.Discover()
            params = {
                "page": 1,
                "primary_release_date.gte": dr["gte"],
                "primary_release_date.lte": dr["lte"]
            }
            try:
                first_page_response = discover.movie(**params)
            except Exception as e:
                print(f"Error fetching first page for date range {dr}: {e}")
                continue

            total_pages = first_page_response.get("total_pages", 1)
            if total_pages > 500:
                total_pages = 500
            print(f"  Total pages in this range: {total_pages}")

            # Process each page in this date range
            for page in range(1, total_pages + 1):
                if movies_processed >= 100:
                    break  # Exit if we've processed 100 movies
                params["page"] = page
                print(f"  Processing page {page}/{total_pages} for range {dr['gte']} to {dr['lte']}...")
                try:
                    response = discover.movie(**params)
                except Exception as e:
                    print(f"  Error on page {page}: {e}")
                    continue

                for movie in response.get("results", []):
                    if movies_processed >= 100:
                        break  # Exit inner loop if limit reached
                    movie_id = movie.get("id")
                    details = get_movie_details(movie_id)
                    if details is None:
                        continue

                    # Write single-valued movie data
                    main_data = process_main_data(details)
                    main_writer.writerow(main_data)

                    # Write collection data (if any)
                    collection_data = process_collection(details)
                    if collection_data:
                        coll_writer.writerow(collection_data)

                    # Write multi-valued data into their respective CSV files
                    for row in process_genres(details):
                        genres_writer.writerow(row)
                    for row in process_production_companies(details):
                        prod_comp_writer.writerow(row)
                    for row in process_production_countries(details):
                        prod_countries_writer.writerow(row)
                    for row in process_spoken_languages(details):
                        spoken_writer.writerow(row)

                    movies_processed += 1
                    print(f"  Processed {movies_processed} movies so far.")

                    # Pause briefly to respect API rate limits
                    time.sleep(0.25)

                if movies_processed >= 100:
                    break  # Exit page loop if limit reached

if __name__ == "__main__":
    main()


Processing movies released from 1900-01-01 to 1950-12-31
  Total pages in this range: 500
  Processing page 1/500 for range 1900-01-01 to 1950-12-31...
  Processed 1 movies so far.
  Processed 2 movies so far.
  Processed 3 movies so far.
  Processed 4 movies so far.
  Processed 5 movies so far.


KeyboardInterrupt: 

 parallelise with threadpool executors?

In [ ]:
import csv
import time
import datetime
import tmdbsimple as tmdb
import threading
import concurrent.futures

tmdb.API_KEY = '8db08b812689d86a7bf90611bf5a1eee'

# -----------------------------
# Data Processing Functions
# -----------------------------
def get_movie_details(movie_id):
    """Fetch detailed information for a movie by its ID."""
    movie = tmdb.Movies(movie_id)
    try:
        details = movie.info()
        return details
    except Exception as e:
        print(f"Error retrieving movie {movie_id}: {e}")
        return None

def process_main_data(details):
    """Extract single-valued (non-normalized) fields for the main movies table."""
    return {
        "id": details.get("id"),
        "adult": details.get("adult"),
        "backdrop_path": details.get("backdrop_path"),
        "budget": details.get("budget"),
        "homepage": details.get("homepage"),
        "imdb_id": details.get("imdb_id"),
        "original_language": details.get("original_language"),
        "original_title": details.get("original_title"),
        "overview": details.get("overview"),
        "popularity": details.get("popularity"),
        "poster_path": details.get("poster_path"),
        "release_date": details.get("release_date"),
        "revenue": details.get("revenue"),
        "runtime": details.get("runtime"),
        "status": details.get("status"),
        "tagline": details.get("tagline"),
        "title": details.get("title"),
        "video": details.get("video"),
        "vote_average": details.get("vote_average"),
        "vote_count": details.get("vote_count")
    }

def process_collection(details):
    """Extract collection info (belongs_to_collection) into its own record."""
    coll = details.get("belongs_to_collection")
    if coll:
        return {
            "movie_id": details.get("id"),
            "collection_id": coll.get("id"),
            "collection_name": coll.get("name")
        }
    return None

def process_genres(details):
    """Extract the list of genres for a movie."""
    genres_list = details.get("genres", [])
    rows = []
    movie_id = details.get("id")
    for genre in genres_list:
        rows.append({
            "movie_id": movie_id,
            "genre_id": genre.get("id"),
            "genre_name": genre.get("name")
        })
    return rows

def process_production_companies(details):
    """Extract production companies for a movie."""
    companies = details.get("production_companies", [])
    rows = []
    movie_id = details.get("id")
    for comp in companies:
        rows.append({
            "movie_id": movie_id,
            "company_id": comp.get("id"),
            "company_name": comp.get("name"),
            "origin_country": comp.get("origin_country")
        })
    return rows

def process_production_countries(details):
    """Extract production countries for a movie."""
    countries = details.get("production_countries", [])
    rows = []
    movie_id = details.get("id")
    for country in countries:
        rows.append({
            "movie_id": movie_id,
            "iso_3166_1": country.get("iso_3166_1"),
            "country_name": country.get("name")
        })
    return rows

def process_spoken_languages(details):
    """Extract spoken languages for a movie."""
    languages = details.get("spoken_languages", [])
    rows = []
    movie_id = details.get("id")
    for lang in languages:
        rows.append({
            "movie_id": movie_id,
            "iso_639_1": lang.get("iso_639_1"),
            "language_name": lang.get("name")
        })
    return rows

In [ ]:
# -----------------------------
# Helper: Process a Single Movie
# -----------------------------
def process_movie(movie, main_writer, coll_writer, genres_writer,
                  prod_comp_writer, prod_countries_writer, spoken_writer, lock):
    """
    Process a single movie: fetch details, write data to CSV (within a lock),
    and then sleep to help respect rate limits.
    """
    movie_id = movie.get("id")
    details = get_movie_details(movie_id)
    if details is None:
        return
    with lock:
        main_data = process_main_data(details)
        main_writer.writerow(main_data)

        collection_data = process_collection(details)
        if collection_data:
            coll_writer.writerow(collection_data)

        for row in process_genres(details):
            genres_writer.writerow(row)
        for row in process_production_companies(details):
            prod_comp_writer.writerow(row)
        for row in process_production_countries(details):
            prod_countries_writer.writerow(row)
        for row in process_spoken_languages(details):
            spoken_writer.writerow(row)
    # Sleep to respect API rate limits.
    time.sleep(0.25)

In [ ]:
# -----------------------------
# Recursive Date Range Processing with Parallelization
# -----------------------------
def process_date_range(gte, lte, main_writer, coll_writer, genres_writer,
                       prod_comp_writer, prod_countries_writer, spoken_writer,
                       flush_files, lock):
    """
    Process movies released between gte and lte.
    If the TMDB discover query returns 500 or more pages, split the range.
    For each page, process movies in parallel and flush file buffers after every 500 pages.
    """
    print(f"\nProcessing movies released from {gte} to {lte}")
    discover = tmdb.Discover()
    params = {
        "page": 1,
        "primary_release_date.gte": gte,
        "primary_release_date.lte": lte
    }

    try:
        first_page_response = discover.movie(**params)
    except Exception as e:
        print(f"Error fetching first page for range {gte} to {lte}: {e}")
        return

    total_pages = first_page_response.get("total_pages", 1)
    print(f"  Total pages in this range: {total_pages}")

    # If we hit the 500-page cap, split the range.
    if total_pages >= 500:
        start_date = datetime.datetime.strptime(gte, "%Y-%m-%d")
        end_date = datetime.datetime.strptime(lte, "%Y-%m-%d")
        if start_date >= end_date:
            print("  Date range too narrow to split further. Processing with current cap.")
        else:
            mid_date = start_date + (end_date - start_date) / 2
            mid_str = mid_date.strftime("%Y-%m-%d")
            print(f"  Splitting range: {gte} to {mid_str} and {mid_str} to {lte}")
            process_date_range(gte, mid_str, main_writer, coll_writer, genres_writer,
                               prod_comp_writer, prod_countries_writer, spoken_writer,
                               flush_files, lock)
            process_date_range(mid_str, lte, main_writer, coll_writer, genres_writer,
                               prod_comp_writer, prod_countries_writer, spoken_writer,
                               flush_files, lock)
            return

    pages_processed = 0
    # Process each page in the current date range.
    for page in range(1, total_pages + 1):
        params["page"] = page
        print(f"  Processing page {page}/{total_pages} for range {gte} to {lte}...")
        try:
            response = discover.movie(**params)
        except Exception as e:
            print(f"  Error on page {page}: {e}")
            continue

        movies = response.get("results", [])
        # Process movies in parallel using a ThreadPoolExecutor.
        with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
            futures = [executor.submit(process_movie, movie, main_writer, coll_writer,
                                         genres_writer, prod_comp_writer, prod_countries_writer,
                                         spoken_writer, lock)
                       for movie in movies]
            # Wait for all submitted tasks to complete.
            concurrent.futures.wait(futures)

        pages_processed += 1
        # Flush file buffers after every 500 pages.
        if pages_processed % 500 == 0:
            for file_obj in flush_files.values():
                file_obj.flush()
            print(f"  Processed {pages_processed} pages; file buffers flushed.")

In [ ]:
# -----------------------------
# Main ETL Process
# -----------------------------
def main():
    # Define output CSV filenames.
    main_file = "tmdb_movies_main.csv"
    collection_file = "tmdb_movie_collection.csv"
    genres_file = "tmdb_movie_genres.csv"
    production_companies_file = "tmdb_movie_production_companies.csv"
    production_countries_file = "tmdb_movie_production_countries.csv"
    spoken_languages_file = "tmdb_movie_spoken_languages.csv"

    # Define CSV headers for each file.
    main_headers = [
        "id", "adult", "backdrop_path", "budget", "homepage", "imdb_id",
        "original_language", "original_title", "overview", "popularity",
        "poster_path", "release_date", "revenue", "runtime", "status",
        "tagline", "title", "video", "vote_average", "vote_count"
    ]
    collection_headers = ["movie_id", "collection_id", "collection_name"]
    genres_headers = ["movie_id", "genre_id", "genre_name"]
    production_companies_headers = ["movie_id", "company_id", "company_name", "origin_country"]
    production_countries_headers = ["movie_id", "iso_3166_1", "country_name"]
    spoken_languages_headers = ["movie_id", "iso_639_1", "language_name"]

    # Open CSV files for writing.
    with open(main_file, mode="w", newline="", encoding="utf-8") as main_csv, \
         open(collection_file, mode="w", newline="", encoding="utf-8") as coll_csv, \
         open(genres_file, mode="w", newline="", encoding="utf-8") as genres_csv, \
         open(production_companies_file, mode="w", newline="", encoding="utf-8") as prod_comp_csv, \
         open(production_countries_file, mode="w", newline="", encoding="utf-8") as prod_countries_csv, \
         open(spoken_languages_file, mode="w", newline="", encoding="utf-8") as spoken_csv:

        main_writer = csv.DictWriter(main_csv, fieldnames=main_headers)
        coll_writer = csv.DictWriter(coll_csv, fieldnames=collection_headers)
        genres_writer = csv.DictWriter(genres_csv, fieldnames=genres_headers)
        prod_comp_writer = csv.DictWriter(prod_comp_csv, fieldnames=production_companies_headers)
        prod_countries_writer = csv.DictWriter(prod_countries_csv, fieldnames=production_countries_headers)
        spoken_writer = csv.DictWriter(spoken_csv, fieldnames=spoken_languages_headers)

        # Write headers to each CSV file.
        main_writer.writeheader()
        coll_writer.writeheader()
        genres_writer.writeheader()
        prod_comp_writer.writeheader()
        prod_countries_writer.writeheader()
        spoken_writer.writeheader()

        # Dictionary of file objects for flushing.
        flush_files = {
            "main": main_csv,
            "collection": coll_csv,
            "genres": genres_csv,
            "prod_companies": prod_comp_csv,
            "prod_countries": prod_countries_csv,
            "spoken": spoken_csv
        }

        # Create a lock for thread-safe CSV writing.
        lock = threading.Lock()

        # Define the full date range: from 1900-01-01 to today.
        start_date = "1900-01-01"
        today_str = datetime.datetime.today().strftime("%Y-%m-%d")
        process_date_range(start_date, today_str,
                           main_writer, coll_writer, genres_writer,
                           prod_comp_writer, prod_countries_writer, spoken_writer,
                           flush_files, lock)

        # Final flush after all processing is complete.
        for file_obj in flush_files.values():
            file_obj.flush()
        print(f"\nCompleted processing movies from {start_date} to {today_str}. Files flushed to disk.")

if __name__ == "__main__":
    main()

###EXTRACTION ACTORS AND DIRECTORS:###

In [ ]:
!pip install tmdbsimple
import tmdbsimple as tmdb
import pandas as pd
import time

tmdb.API_KEY = '73df6e10f160a8dc3afc422d2ddf7bb3'

# Function to get actor/director details
def get_person_details(person_id):
    try:
        person = tmdb.People(person_id)
        details = person.info()
        return {
            'ID': person_id,
            'Name': details.get('name', 'N/A'),
            'Date of Birth': details.get('birthday', 'N/A'),
            'Nationality': details.get('place_of_birth', 'N/A'),
            'Known For': [job['title'] for job in details.get('known_for', [])],
            'Biography': details.get('biography', 'N/A')
        }
    except Exception as e:
        print(f"Error fetching details for person ID {person_id}: {e}")
        return None

# Function to get actors and directors from popular movies
def get_actors_directors_from_movies(year, num_movies=10):
    movie_data = []
    discover = tmdb.Discover()
    response = discover.movie(primary_release_year=year, language='en')

    for movie in response['results'][:num_movies]:  # Limit to avoid excessive requests
        movie_id = movie['id']
        credits = tmdb.Movies(movie_id).credits()

        for person in credits.get('cast', [])[:5]:  # Get top 5 actors
            movie_data.append({'Movie': movie['title'], 'Role': 'Actor', 'Person ID': person['id']})

        for person in credits.get('crew', []):
            if person['job'] == 'Director':
                movie_data.append({'Movie': movie['title'], 'Role': 'Director', 'Person ID': person['id']})

        time.sleep(0.5)  # Rate limit handling

    return movie_data

# Fetch actor & director data from movies of a given year
movies_data = get_actors_directors_from_movies(2023, num_movies=5)

# Get details for all actors/directors found
people_data = []
for entry in movies_data:
    person_details = get_person_details(entry['Person ID'])
    if person_details:
        person_details['Movie'] = entry['Movie']
        person_details['Role'] = entry['Role']
        people_data.append(person_details)
    time.sleep(0.5)

# Convert to DataFrame
df_people = pd.DataFrame(people_data)

from IPython.display import display

# Display the DataFrame
display(df_people)

In [ ]:
!pip install tmdbsimple
import tmdbsimple as tmdb
import pandas as pd
import time

tmdb.API_KEY = '73df6e10f160a8dc3afc422d2ddf7bb3'

# Function to get actor/director details
def get_person_details(person_id):
    """Fetch details of an actor/director by ID"""
    try:
        person = tmdb.People(person_id)
        details = person.info()
        credits = person.combined_credits()  # Get movie & TV credits

        return {
            'ID': person_id,
            'Name': details.get('name', 'N/A'),
            'Date of Birth': details.get('birthday', 'N/A'),
            'Nationality': details.get('place_of_birth', 'N/A'),
            'Known For': [work['title'] for work in credits.get('cast', [])[:10]],  # Get more movies
            'Biography': details.get('biography', 'N/A')
        }
    except Exception as e:
        print(f"Error fetching details for person ID {person_id}: {e}")
        return None

# Function to get actors & directors from a large set of movies
def get_actors_directors_from_movies(start_year, end_year, max_movies_per_year=50):
    """Get a large list of actors and directors from many movies"""
    movie_data = []

    for year in range(start_year, end_year + 1):
        print(f"Fetching movies from {year}...")
        discover = tmdb.Discover()
        response = discover.movie(primary_release_year=year, language='en')

        for movie in response['results'][:max_movies_per_year]:  # Get more movies per year
            movie_id = movie['id']
            credits = tmdb.Movies(movie_id).credits()

            # Extract **all** actors
            for person in credits.get('cast', []):
                movie_data.append({'Movie': movie['title'], 'Role': 'Actor', 'Person ID': person['id']})

            # Extract **all** directors
            for person in credits.get('crew', []):
                if person['job'] == 'Director':
                    movie_data.append({'Movie': movie['title'], 'Role': 'Director', 'Person ID': person['id']})

            time.sleep(0.2)  # Reduce delay for high-volume scraping

    return movie_data

# Function to scrape **thousands** of popular people
def get_popular_people(max_pages=20):
    """Scrape as many popular actors & directors as possible"""
    popular_people = []
    people_api = tmdb.People()

    for page in range(1, max_pages + 1):
        print(f"Fetching page {page} of popular people...")
        response = people_api.popular(page=page)

        for person in response['results']:
            popular_people.append({'Name': person['name'], 'Person ID': person['id']})

        time.sleep(0.2)

    return popular_people

# Step 1: Scrape **movies from multiple years**
movies_data = get_actors_directors_from_movies(2000, 2024, max_movies_per_year=100)  # Increase range for max data

# Step 2: Scrape **thousands of popular people**
popular_people = get_popular_people(max_pages=50)  # Increase pages for max data

# Step 3: Get details for **all** actors & directors
all_people_data = []

# From **Movies**
for entry in movies_data:
    person_details = get_person_details(entry['Person ID'])
    if person_details:
        person_details['Movie'] = entry['Movie']
        person_details['Role'] = entry['Role']
        all_people_data.append(person_details)
    time.sleep(0.1)  # Fast scraping

# From **Popular People**
for entry in popular_people:
    person_details = get_person_details(entry['Person ID'])
    if person_details:
        person_details['Movie'] = 'N/A'  # No specific movie for popular people
        person_details['Role'] = 'N/A'
        all_people_data.append(person_details)
    time.sleep(0.1)

# Convert to DataFrame
df_people = pd.DataFrame(all_people_data)

# Save to CSV for BIG data processing
df_people.to_csv("actors_directors_data.csv", index=False)

# Display data
from IPython.display import display
display(df_people)

In [ ]:

#extracting movies from known for column
# Convert string representation of lists to actual lists using ast.literal_eval
df_people['Known For'] = df['Known For'].apply(ast.literal_eval)

# Create a new dataframe by exploding the Known For column
new_df = df[['Name', 'Known For', 'Role']].explode('Known For')

# Rename the columns
new_df = new_df.rename(columns={'Known For': 'Movie/TV Show'})

# Reset the index
new_df = new_df.reset_index(drop=True)

# Display the first 15 rows of the new dataframe
print("New DataFrame (first 15 rows):")
print(new_df.head(15))

# Display some basic statistics
print("\nDataset Statistics:")
print(f"Total number of entries: {len(new_df)}")
print(f"Number of unique people: {new_df['Name'].nunique()}")
print(f"Number of unique movies/shows: {new_df['Movie/TV Show'].nunique()}")

# Save the new dataframe to a CSV file
new_df.to_csv('person_movie_role.csv', index=False)
print("\nNew CSV file 'person_movie_role.csv' has been created!")

###EXTRACTION TV SHOWS:####

In [ ]:
def showBasic(start_year, end_year):
  shows_data = []
  for year in range(start_year, end_year + 1):
    print(f"Fetching data for year {year}...")
    discover = tmdb.Discover()
    response = discover.tv(first_air_date_year=year, language='en')

    for show in response['results']:
      shows_data.append({
          'ID': show['id'],
          'Name': show['name'],
          'Original Name': show['original_name']
          })

    total_pages = response['total_pages']
    for page in range(2, min(total_pages + 1, 501)):
      response = discover.tv(first_air_date_year=year, page=page, language='en')
      for show in response['results']:
        shows_data.append({
            'ID': show['id'],
            'Name': show['name'],
            'Original Name': show['original_name']
            })

    time.sleep(0.5)
  return pd.DataFrame(shows_data)

In [ ]:
def showBasicDownload():
  df1_1 = showBasic(start_year=1960, end_year=1970)
  df1_2 = showBasic(start_year=1971, end_year=1980)
  df1_3 = showBasic(start_year=1981, end_year=1990)
  df1_4 = showBasic(start_year=1991, end_year=2000)
  df1_5 = showBasic(start_year=2001, end_year=2010)
  df1_6 = showBasic(start_year=2011, end_year=2015)
  df1_7 = showBasic(start_year=2016, end_year=2019)
  df1_8 = showBasic(start_year=2020, end_year=2022)
  df1_9 = showBasic(start_year=2023, end_year=2025)

  return pd.concat([df1_1, df1_2, df1_3, df1_4, df1_5, df1_6, df1_7, df1_8, df1_9], ignore_index=True)

In [ ]:
df1 = showBasicDownload()

In [ ]:
df1.to_csv('TV_Show(Basic).csv', index=False)

###SHOW OVERVIEW ID SINOPSIS###

In [ ]:
def showOverview(start_year, end_year):
  shows_data = []
  for year in range(start_year, end_year + 1):
    print(f"Fetching data for year {year}...")
    discover = tmdb.Discover()
    response = discover.tv(first_air_date_year=year, language='en')
    for show in response['results']:
      shows_data.append({
          'ID': show['id'],
          'Synopsis': show['overview']
          })

    total_pages = response['total_pages']
    for page in range(2, min(total_pages + 1, 501)):
      response = discover.tv(first_air_date_year=year, page=page, language='en')
      for show in response['results']:
        shows_data.append({
            'ID': show['id'],
            'Synopsis': show['overview']
            })

    time.sleep(0.5)
  return pd.DataFrame(shows_data)

In [ ]:
df1_1 = showOverview(start_year=1960, end_year=1970)
  df1_2 = showOverview(start_year=1971, end_year=1980)
  df1_3 = showOverview(start_year=1981, end_year=1990)
  df1_4 = showOverview(start_year=1991, end_year=2000)
  df1_5 = showOverview(start_year=2001, end_year=2010)
  df1_6 = showOverview(start_year=2011, end_year=2015)
  df1_7 = showOverview(start_year=2016, end_year=2019)
  df1_8 = showOverview(start_year=2020, end_year=2022)
  df1_9 = showOverview(start_year=2023, end_year=2025)

  return pd.concat([df1_1, df1_2, df1_3, df1_4, df1_5, df1_6, df1_7, df1_8, df1_9], ignore_index=True)

In [ ]:
df2 = showOverviewDownload()

In [ ]:
df2.to_csv('TV_Show(Overview).csv', index=False)

###SHOWS SEASONS AND EPISODES ###

In [ ]:
def showSeasEps(start_year, end_year):
    shows_data = []

    for year in range(start_year, end_year + 1):
        print(f"Fetching data for year {year}...")

        discover = tmdb.Discover()
        response = discover.tv(first_air_date_year=year, language='en')

        for show in response['results']:
            try:
                show_details = tmdb.TV(show['id']).info()
                shows_data.append({
                    'ID': show['id'],
                    'Seasons': show_details.get('number_of_seasons', 0),
                    'Episodes': show_details.get('number_of_episodes', 0)
                })
            except HTTPError as e:
                print(f"Error fetching data for show ID {show['id']}: {e}")
                shows_data.append({
                    'ID': show['id'],
                    'Seasons': "Unknown (Error)",
                    'Episodes': "Unknown (Error)"
                })

        total_pages = response['total_pages']
        for page in range(2, min(total_pages + 1, 501)):
            response = discover.tv(first_air_date_year=year, page=page, language='en')
            for show in response['results']:
                try:
                    show_details = tmdb.TV(show['id']).info()
                    shows_data.append({
                        'ID': show['id'],
                        'Seasons': show_details.get('number_of_seasons', 0),
                        'Episodes': show_details.get('number_of_episodes', 0)
                    })
                except HTTPError as e:
                    print(f"Error fetching data for show ID {show['id']}: {e}")
                    shows_data.append({
                        'ID': show['id'],
                        'Seasons': "Unknown (Error)",
                        'Episodes': "Unknown (Error)"
                    })

        time.sleep(0.75)

    return pd.DataFrame(shows_data)

In [ ]:
def showSeasEpsDownload():
  df1_1 = showSeasEps(start_year=1960, end_year=1970)
  df1_2 = showSeasEps(start_year=1971, end_year=1980)
  df1_3 = showSeasEps(start_year=1981, end_year=1990)
  df1_4 = showSeasEps(start_year=1991, end_year=2000)
  df1_5 = showSeasEps(start_year=2001, end_year=2010)
  df1_6 = showSeasEps(start_year=2011, end_year=2015)
  df1_7 = showSeasEps(start_year=2016, end_year=2019)
  df1_8 = showSeasEps(start_year=2020, end_year=2022)
  df1_9 = showSeasEps(start_year=2023, end_year=2025)

  return pd.concat([df1_1, df1_2, df1_3, df1_4, df1_5, df1_6, df1_7, df1_8, df1_9], ignore_index=True)

In [ ]:
df3 = showSeasEpsDownload()

In [ ]:
df3.to_csv('TV_Show(SeasEps).csv', index=False)

###Show Production (ID, Exectuive Producers, Production Companies, Networks)###

In [ ]:
def showProduction(start_year, end_year):
    shows_data = []

    for year in range(start_year, end_year + 1):
        print(f"Fetching data for year {year}...")

        discover = tmdb.Discover()
        response = discover.tv(first_air_date_year=year, language='en')

        for show in response['results']:
            try:
                show_details = tmdb.TV(show['id']).info()
                credits = tmdb.TV(show['id']).credits()
                executive_producers = [member['name'] for member in credits.get('crew', []) if member['job'] == 'Executive Producer']
                production_companies = [company['name'] for company in show_details.get('production_companies', [])]
                networks = [network['name'] for network in show_details.get('networks', [])]

                shows_data.append({
                    'ID': show['id'],
                    'Executive Producers': executive_producers,
                    'Production Companies': production_companies,
                    'Networks': networks
                })
            except HTTPError as e:
                print(f"Error fetching data for show ID {show['id']}: {e}")
                shows_data.append({
                    'ID': show['id'],
                    'Executive Producers': [],
                    'Production Companies': [],
                    'Networks': []
                })

        total_pages = response['total_pages']
        for page in range(2, min(total_pages + 1, 501)):
            response = discover.tv(first_air_date_year=year, page=page, language='en')
            for show in response['results']:
                try:
                    show_details = tmdb.TV(show['id']).info()
                    credits = tmdb.TV(show['id']).credits()
                    executive_producers = [member['name'] for member in credits.get('crew', []) if member['job'] == 'Executive Producer']
                    production_companies = [company['name'] for company in show_details.get('production_companies', [])]
                    networks = [network['name'] for network in show_details.get('networks', [])]
                    shows_data.append({
                        'ID': show['id'],
                        'Executive Producers': executive_producers,
                        'Production Companies': production_companies,
                        'Networks': networks
                    })
                except HTTPError as e:
                    print(f"Error fetching data for show ID {show['id']}: {e}")
                    shows_data.append({
                        'ID': show['id'],
                        'Executive Producers': [],
                        'Production Companies': [],
                        'Networks': []
                    })

        time.sleep(0.75)

    return pd.DataFrame(shows_data)

In [ ]:
def showProductionDownload():
  df1_1 = showProduction(start_year=1960, end_year=1970)
  df1_2 = showProduction(start_year=1971, end_year=1980)
  df1_3 = showProduction(start_year=1981, end_year=1990)
  df1_4 = showProduction(start_year=1991, end_year=2000)
  df1_5 = showProduction(start_year=2001, end_year=2010)
  df1_6 = showProduction(start_year=2011, end_year=2015)
  df1_7 = showProduction(start_year=2016, end_year=2019)
  df1_8 = showProduction(start_year=2020, end_year=2022)
  df1_9 = showProduction(start_year=2023, end_year=2025)

  return pd.concat([df1_1, df1_2, df1_3, df1_4, df1_5, df1_6, df1_7, df1_8, df1_9], ignore_index=True)

In [ ]:
df4 = showProductionDownload()

In [ ]:
df4.to_csv('TV_Show(Production).csv', index=False)

###Show Ratings (ID, Popularity Score, Genres)###

In [ ]:
def showRatings(start_year, end_year):
  shows_data = []
  for year in range(start_year, end_year + 1):
    print(f"Fetching data for year {year}...")
    discover = tmdb.Discover()
    response = discover.tv(first_air_date_year=year, language='en')

    for show in response['results']:
      show_id = show['id']
      show_details = tmdb.TV(show_id)
      show_details_info = show_details.info()
      genres = [genre['name'] for genre in show_details_info.get('genres', [])]

      shows_data.append({
          'ID': show['id'],
          'Popularity Score': show.get('popularity', 'Unknown'),
          'Genres': ', '.join(genres)
          })

    total_pages = response['total_pages']
    for page in range(2, min(total_pages + 1, 501)):
      response = discover.tv(first_air_date_year=year, page=page, language='en')
      for show in response['results']:
        show_id = show['id']
        show_details = tmdb.TV(show_id)
        show_details_info = show_details.info()
        genres = [genre['name'] for genre in show_details_info.get('genres', [])]

        shows_data.append({
            'ID': show['id'],
            'Popularity Score': show.get('popularity', 'Unknown'),
            'Genres': ', '.join(genres)
            })

    time.sleep(0.5)
  return pd.DataFrame(shows_data)

In [ ]:
def showRatingsDownload():
  df1_1 = showRatings(start_year=1960, end_year=1970)
  df1_2 = showRatings(start_year=1971, end_year=1980)
  df1_3 = showRatings(start_year=1981, end_year=1990)
  df1_4 = showRatings(start_year=1991, end_year=2000)
  df1_5 = showRatings(start_year=2001, end_year=2010)
  df1_6 = showRatings(start_year=2011, end_year=2015)
  df1_7 = showRatings(start_year=2016, end_year=2019)
  df1_8 = showRatings(start_year=2020, end_year=2022)
  df1_9 = showRatings(start_year=2023, end_year=2025)

  return pd.concat([df1_1, df1_2, df1_3, df1_4, df1_5, df1_6, df1_7, df1_8, df1_9], ignore_index=True)

In [ ]:
df5 = showRatingsDownload()

In [ ]:
df5.to_csv('TV_Show(Ratings).csv', index=False)

###Show Airings (ID, First Aired Date, Last Aired Date, Status)###

In [ ]:
def showAirings(start_year, end_year):
  shows_data = []
  for year in range(start_year, end_year + 1):
    print(f"Fetching data for year {year}...")
    discover = tmdb.Discover()
    response = discover.tv(first_air_date_year=year, language='en')

    for show in response['results']:
      show_id = show['id']
      show_details = tmdb.TV(show_id)
      show_details_info = show_details.info()

      shows_data.append({
          'ID': show['id'],
          'First Aired Date': show_details_info.get('first_air_date', 'Unknown'),
          'Last Aired Date': show_details_info.get('last_air_date', 'Unknown'),
          'Status': show_details_info.get('status', 'Unknown')
          })

    total_pages = response['total_pages']
    for page in range(2, min(total_pages + 1, 501)):
      response = discover.tv(first_air_date_year=year, page=page, language='en')
      for show in response['results']:
        show_id = show['id']
        show_details = tmdb.TV(show_id)
        show_details_info = show_details.info()

        shows_data.append({
            'ID': show['id'],
            'First Aired Date': show_details_info.get('first_air_date', 'Unknown'),
            'Last Aired Date': show_details_info.get('last_air_date', 'Unknown'),
            'Status': show_details_info.get('status', 'Unknown')
            })

    time.sleep(0.75)
  return pd.DataFrame(shows_data)

In [ ]:
def showAiringsDownload():
  df1_1 = showAirings(start_year=1960, end_year=1970)
  df1_2 = showAirings(start_year=1971, end_year=1980)
  df1_3 = showAirings(start_year=1981, end_year=1990)
  df1_4 = showAirings(start_year=1991, end_year=2000)
  df1_5 = showAirings(start_year=2001, end_year=2005)
  df1_6 = showAirings(start_year=2006, end_year=2010)
  df1_7 = showAirings(start_year=2011, end_year=2015)
  df1_8 = showAirings(start_year=2016, end_year=2019)
  df1_9 = showAirings(start_year=2020, end_year=2022)
  df1_10 = showAirings(start_year=2023, end_year=2025)

  return pd.concat([df1_1, df1_2, df1_3, df1_4, df1_5, df1_6, df1_7, df1_8, df1_9, df1_10], ignore_index=True)

In [ ]:
df6 = showAiringsDownload()

In [ ]:
df6.to_csv('TV_Show(Airings).csv', index=False)

###DATA CLEANING TV SHOWS##


In [ ]:
df7 = df4

In [ ]:
def clean_list_column(column):
    return column.str.strip("[]").str.replace("'", "").str.replace('"', '')

In [ ]:
columns_to_clean = ["Executive Producers", "Production Companies", "Networks"]
df7[columns_to_clean] = df7[columns_to_clean].apply(clean_list_column)

In [ ]:
df8.to_csv('TV_Show(Production_Cleaned).csv', index=False)

###MOVIES and TV SHOWS EXTRACTION:

In [ ]:
import tmdbsimple as tmdb
import json
import requests
import pandas as pd

# Your TMDb API key
tmdb.API_KEY = '8db08b812689d86a7bf90611bf5a1eee'
tmdb.REQUESTS_SESSION = requests.Session()


# Function to inspect movie data (recommendations, reviews)
def inspect_movie_data(movie_id):
    movie = tmdb.Movies(movie_id)

    # Fetch recommendations
    recommendations = movie.recommendations()

    # Extract movie recommendations data
    movie_recommendations = []
    for item in recommendations.get('results', []):
        movie_recommendations.append({
            'Movie ID': item.get('id'),
            'Title': item.get('title'),
            'Overview': item.get('overview'),
            'Release Date': item.get('release_date'),
            'Rating': item.get('vote_average')
        })

    # Fetch reviews
    reviews = movie.reviews()

    # Extract movie reviews data
    movie_reviews = []
    for item in reviews.get('results', []):
        movie_reviews.append({
            'Review ID': item.get('id'),
            'Author': item.get('author'),
            'Content': item.get('content'),
            'Rating': item.get('author_details', {}).get('rating'),
            'Review Date': item.get('created_at')
        })

    # Create DataFrames
    recommendations_df = pd.DataFrame(movie_recommendations)
    reviews_df = pd.DataFrame(movie_reviews)

    return recommendations_df, reviews_df


# Function to inspect TV series data (recommendations, reviews)
def inspect_tv_series_data(tv_id):
    tv_series = tmdb.TV(tv_id)

    # Fetch recommendations
    recommendations = tv_series.recommendations()

    # Extract TV series recommendations data
    tv_recommendations = []
    for item in recommendations.get('results', []):
        tv_recommendations.append({
            'TV Series ID': item.get('id'),
            'Title': item.get('name'),
            'Overview': item.get('overview'),
            'First Air Date': item.get('first_air_date'),
            'Rating': item.get('vote_average')
        })

    # Fetch reviews
    reviews = tv_series.reviews()

    # Extract TV series reviews data
    tv_reviews = []
    for item in reviews.get('results', []):
        tv_reviews.append({
            'Review ID': item.get('id'),
            'Author': item.get('author'),
            'Content': item.get('content'),
            'Rating': item.get('author_details', {}).get('rating'),
            'Review Date': item.get('created_at')
        })

    # Create DataFrames
    recommendations_df = pd.DataFrame(tv_recommendations)
    reviews_df = pd.DataFrame(tv_reviews)

    return recommendations_df, reviews_df



def print_json_formatted(json_data):
    """Prints JSON data in a formatted way."""
    formatted_json = json.dumps(json_data, indent=4)
    print(formatted_json)




# Inspect Movie Data
movie_recommendations_df, movie_reviews_df = inspect_movie_data(movie_id)
print("Movie Recommendations:")
print(movie_recommendations_df.head())  # Display first 5 rows of movie recommendations
print("\nMovie Reviews:")
print(movie_reviews_df.head())  # Display first 5 rows of movie reviews

# Inspect TV Series Data
tv_recommendations_df, tv_reviews_df = inspect_tv_series_data(tv_id)
print("\nTV Series Recommendations:")
print(tv_recommendations_df.head())  # Display first 5 rows of TV series recommendations
print("\nTV Series Reviews:")
print(tv_reviews_df.head())  # Display first 5 rows of TV series reviews
print(movie_recommendations_df.columns)
# Save Movie DataFrames to CSV
movie_recommendations_df.to_csv('movie_recommendations.csv', index=False)
movie_reviews_df.to_csv('movie_reviews.csv', index=False)

# Save TV Series DataFrames to CSV
tv_recommendations_df.to_csv('tv_recommendations.csv', index=False)
tv_reviews_df.to_csv('tv_reviews.csv', index=False)

print("CSV files saved successfully.")

